# Ejemplo CodeGen decoder

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# cargamos el tokenizador y el modelo de CodeGen, algo antiguo pero sirve 
# para mostrar el por qué hay que usar SFT
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
# lo cargamos en bfloat16 para ahorrar memoria
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono", 
                                            trust_remote_code=True, revision="main", 
                                            device_map="auto",
                                            dtype=torch.bfloat16)

In [ ]:
# prompt que usaremos para generar código
PROMPT = """### Instruction:
{instruction}

### Response:
{response}"""

In [ ]:
from transformers import set_seed
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
set_seed(42)

# generamos el input
text = PROMPT.format(
    instruction="compute the mean of a numpy array",
    response=""
)
# lo tokenizamos
input_ids = tokenizer(text, return_tensors="pt").input_ids
# generamos la continuación
generated_ids = model.generate(input_ids.cuda(), max_length=128, do_sample=True, temperature=0.1,
                               eos_token_id=tokenizer.eos_token_id)
# mostramos el resultado
print()
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# si nos vamos a uno más grande Salesforce/codegen2-1B, obtendremos algo mejor pero tendremos un problema similar
